In [1]:
import numpy as np
from ipynb.fs.full.Critical_Section import *
import re


In [2]:
'''
Converts the VC given in string to an array

Input:
vc: Vector clock 
[(0:3) (1:0) (2:0) (3:0) (4:3)]
n: Number of threads 
5

Output:
[3. 0. 0. 0. 3.]
'''
def generate_vc_array(vc,n):
    # find (0:3)
    r1 = re.findall("\(\d+:\d+\)",vc)
    vc_array = np.zeros((len(r1)))
    for exp in r1:
        if int(exp[1])>=0:
            vc_array[int(exp[1])] = int(exp[3])
    return vc_array

'''
Returns array of the VC's for each event which consists of tuple of (VC,VC_R,VC_W)


Input:
events: Events array
[event_raw(vc_w='[(0:0) (1:1) (2:0) (3:1) (4:1)]', vc_r='[(0:0) (1:0) (2:3)]', vc='[(0:3) (1:0) (2:0) (3:0) (4:3)]', tid=4, e_type=1, var_name='test/Figure1_x'), 
event_raw(vc_w='[(0:0) (1:1) (2:0) (3:1) (4:1)]', vc_r='[(0:0) (1:0) (2:3)]', vc='[(0:3) (1:0) (2:2) (3:3) (4:2)]', tid=3, e_type=1, var_name='test/Figure1_x')
]

n: Number of threads 
5

Output:
[[array([3., 0., 0., 0., 3.]), array([0., 0., 3.]), array([0., 1., 0., 1., 1.])], 
[array([3., 0., 2., 3., 2.]), array([0., 0., 3.]), array([0., 1., 0., 1., 1.])]]

'''

def generate_vc_for_events(events,n):
    vc_for_events = []  
    for event in events:
        vc_array  = generate_vc_array(event.vc,n)
        vc_r_array  = generate_vc_array(event.vc_r,n)
        vc_w_array  = generate_vc_array(event.vc_w,n)
        vc_for_events.append([vc_array,vc_r_array,vc_w_array])
    return vc_for_events
    
'''
pass1 of the algorithm where the events are parsed, CS info is generated, VC for events is parsed.
'''
def pass1(filename):
    events, critical_section_list,n,lock_map = gen_event_cs(filename)
    print(lock_map)
    vc_for_events = generate_vc_for_events(events,n)
    return events,critical_section_list,vc_for_events,n,lock_map    

In [8]:
'''
Checks if the two entries are CP ordered.
'''
def is_cp(cs_index1,cs_index2,cs):
    if(len(list(cs[cs_index1].write_objects.intersection(cs[cs_index2].read_objects)))>0 or 
    len(list(cs[cs_index1].write_objects.intersection(cs[cs_index2].write_objects)))>0 or 
    len(list(cs[cs_index1].read_objects.intersection(cs[cs_index2].write_objects)))>0):
        return True
    
    
# TODO Mark CP and HB matrices at one point only  
'''
Marks 1 for each value of  matrix[arr1][arr2] 
'''
def mark_matrix(matrix,arr1,arr2):
    for val in arr1:
        for val2 in arr2:
            matrix[val][val2]=1
    
'''
Generate CP matrix using rule A in CP paper that the variables must be conflicting for each lock section. 
'''    
def generate_hb_cp_matrix_using_rulea(events,critical_section_list,lock_map):   
    hb_matrix = np.zeros((len(events),len(events)))
    cp_matrix = np.zeros((len(events),len(events)))
    for key, value in lock_map.items():
        for iter_var in range(len(value)):
            for inner_iter_var in range(iter_var+1,len(value)):
                mark_matrix(hb_matrix,critical_section_list[iter_var],critical_section_list[inner_iter_var],inner_iter_var)
                if is_cp(iter_var,inner_iter_var,critical_section_list):
                    mark_matrix(cp_matrix,critical_section_list[iter_var],critical_section_list[inner_iter_var],inner_iter_var)
    return cp_matrix,hb_matrix

'''
Mark Program order. All the events under same thread are both in HB and CP
'''
def mark_po(events,cp_matrix,hb_matrix):
    for i in range(0,len(events)):
        event1_tid = events[i].tid
        for j in range(i+1,len(events)):
            if events[j].tid==event1_tid:
                hb_matrix[i][j] = 1
                cp_matrix[i][j] = 1
    return cp_matrix,hb_matrix
                
def pass2(events,critical_section_list,lock_map):
    cp_matrix,hb_matrix = generate_hb_cp_matrix_using_rulea(generate_hb_cp_matrix)
    cp_matrix,hb_matrix = mark_po(events,cp_matrix,hb_matrix)
    return cp_matrix,hb_matrix


def compute_rule2_rule3(hb_matrix,cp_matrix):
    # start from n - 1 
    for i in reversed(range(hb_matrix.shape[0]-1)):
        for j in (range(hb_matrix.shape[0],i,-1)):
    #         for k in range(j+1,end+1):
              for k in range(end-1,j,-1):
                # print(i,j,k)
                if hb_matrix[i][j] == 1 and cp_matrix[j][k] == 1:
                    hb_matrix[i][k] = 1
                    cp_matrix[i][k] = 1
                elif cp_matrix[i][j] == 1 and hb_matrix[j][k] == 1:
                    hb_matrix[i][k] = 1
                    cp_matrix[i][k] = 1   
    return hb_matrix,cp_matrix

In [5]:
# events, critical_section_list,vc_for_events,no_of_threads,lock_map = pass1("../Figure1.log")
